# 05 - Cohere Interview Question Generation
`Author: Abdlazeez Jimoh`

In [1]:
import json
from pprint import pprint
from typing import Literal, TypedDict

import cohere

In [2]:
COHERE_API_KEY = "YOUR_API_KEY"

In [3]:
class Question(TypedDict):
    question: str
    type: Literal["personal", "role-specific", "behavioral", "situational"]


In [8]:
class CohereQuestionGeneratorAgent:
    def __init__(self):
        super().__init__()
        self.client = cohere.Client(COHERE_API_KEY)
        self.system_prompt = """You are a non-technical interviewer that interviews \
across the following categories:
- personal
- role-specific
- behavioural
- situational

You will be provided with a candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category \
and the questions should be based on the candidate's description.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, \
and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "type": "personal"}},
    {{"question": "<role_specific_question>", "type": "role-specific"}},
    {{"question": "<behavioural_question>", "type": "behavioural"}},
    {{"question": "<situational_question>", "type": "situational"}},
    ...more questions to make up {n_questions} questions
]


===
Candidate Description:
{description}"""

    def __call__(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list[Question] | None: A list of generated interview questions or None if an error occurs.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output = self.client.generate(
                model="command",
                prompt=self.system_prompt.format(n_questions=n_questions, description=description),
                temperature=1,
                max_tokens=1024,
            )
            questions = json.loads(output.generations[0].text)

            return questions
        except Exception as e:
            print(e)
            return None

In [9]:
question_generator = CohereQuestionGeneratorAgent()
questions = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of experience and I'm looking for a new job.",
    5,
)

pprint(questions)

[{'question': 'What motivated you to leave your previous role and seek out a '
              'new opportunity at our company?',
  'type': 'personal'},
 {'question': 'Can you describe a time when you had to manage a heavy '
              'workload? How did you prioritize your tasks?',
  'type': 'role-specific'},
 {'question': 'Tell me about a time when you collaborated with a team to solve '
              'a complex problem. What was your role in this collaboration?',
  'type': 'behavioural'},
 {'question': 'How do you approach learning about and adopting new programming '
              'languages or technologies?',
  'type': 'role-specific'},
 {'question': 'Describe a situation where you had to troubleshoot a '
              'particularly difficult issue. What steps did you take to '
              'identify and resolve the problem?',
  'type': 'behavioural'},
 {'question': 'At your current or previous company, how did you ensure that '
              'you were keeping up-to-date with in

In [10]:
questions = question_generator.run(
    "a data scientist from India. I have 3 years of experience. I've worked on a variety of projects, including a recommendation engine for a large e-commerce company. I've led a team of 5 data scientists and engineers.",
    8
)

pprint(questions)

[{'question': 'Could you tell me about a time when you used creativity to '
              'solve a problem in your work? What was the issue you were '
              'facing and how did your solution improve the situation?',
  'type': 'behavioural'},
 {'question': 'What is the most interesting project you have worked on so far '
              'and why?',
  'type': 'personal'},
 {'question': 'As a Data Scientist, how do you ensure that your team members '
              'have adequate support and resources to complete their projects?',
  'type': 'role-specific'},
 {'question': 'What risks have you encountered in your previous projects? How '
              'did you manage or mitigate these risks?',
  'type': 'behavioural'},
 {'question': 'Describe a time when you had to present your findings to a '
              'non-technical audience. How did you convey your ideas clearly '
              'and effectively?',
  'type': 'behavioural'},
 {'question': 'What is your approach to keeping up to d